**[SQL Home Page](https://www.kaggle.com/learn/intro-to-sql)**

---


# Introduction

[Stack Overflow](https://stackoverflow.com/) is a widely beloved question and answer site for technical questions. You'll probably use it yourself as you keep using SQL (or any programming language). 

Their data is publicly available. What cool things do you think it would be useful for?

Here's one idea:
You could set up a service that identifies the Stack Overflow users who have demonstrated expertise with a specific technology by answering related questions about it, so someone could hire those experts for in-depth help.

In this exercise, you'll write the SQL queries that might serve as the foundation for this type of service.

As usual, run the following cell to set up our feedback system before moving on.

In [1]:
# set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex6 import *
print("Setup is completed")

Using Kaggle's public dataset BigQuery integration.
Setup is completed


Run the next cell to fetch the `stackoverflow` dataset.

In [2]:
# create a "Client" object
from google.cloud import bigquery
client = bigquery.Client()

# construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Exercises

## 1. Explore the Data

Before writing queries or **JOIN** clauses, you'll want to see what tables are available. 

*Hint*: Tab completion is helpful whenever you can't remember a command. Type `client.` and then hit the tab key. Don't forget the period before hitting tab.

In [3]:
# get a list of available tables 
list_of_tables = [table.table_id for table in client.list_tables(dataset)]

# print your answer
print(list_of_tables)

# check your answer
q_1.check()

['badges', 'comments', 'post_history', 'post_links', 'posts_answers', 'posts_moderator_nomination', 'posts_orphaned_tag_wiki', 'posts_privilege_wiki', 'posts_questions', 'posts_tag_wiki', 'posts_tag_wiki_excerpt', 'posts_wiki_placeholder', 'stackoverflow_posts', 'tags', 'users', 'votes']


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# for the solution, uncomment the line below.
# q_1.solution()

<hr/>

## 2. Review Relevant Tables

If you are interested in people who answer questions on a given topic, the `posts_answers` table is a natural place to look. Run the following cell, and look at the output.

In [5]:
# construct a reference to the "posts_answers" table
answers_table_ref = dataset_ref.table("posts_answers")

# API request - fetch the table
answers_table = client.get_table(answers_table_ref)

# preview the first five lines of the "posts_answers" table
client.list_rows(answers_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,59891494,None,<p>Method - 1</p>\n\n<p>It can be resolved by ...,None,None,0,None,2020-01-24 06:38:32.793000+00:00,None,2020-01-24 06:38:32.793000+00:00,None,None,None,None,8101535,59891493,2,0,None,None
1,59891498,None,<p>I hope this will help.</p>\n\n<pre><code>fu...,None,None,0,None,2020-01-24 06:38:38.377000+00:00,None,2020-01-24 06:38:38.377000+00:00,None,None,None,None,8427204,44183708,2,0,None,None
2,59891502,None,<p>Add class in row which you want to calculat...,None,None,0,None,2020-01-24 06:38:45.920000+00:00,None,2020-01-24 06:38:45.920000+00:00,None,None,None,None,6920236,59891186,2,0,None,None
3,59891504,None,"<pre><code>with open('file.txt','r') as f:\n ...",None,None,0,None,2020-01-24 06:38:50.550000+00:00,None,2020-01-24 06:38:50.550000+00:00,None,None,None,None,9851541,59884511,2,0,None,None
4,59891508,None,<p>I could fix the ALB ingress controller issu...,None,None,0,None,2020-01-24 06:39:06.210000+00:00,None,2020-01-24 06:39:06.210000+00:00,None,None,None,None,12599017,59627068,2,0,None,None


It isn't clear yet how to find users who answered questions on any given topic. But `posts_answers` has a `parent_id` column. If you are familiar with the Stack Overflow site, you might figure out that the `parent_id` is the question each post is answering.

Look at `posts_questions` using the cell below.

In [6]:
# construct a reference to the "posts_questions" table
questions_table_ref = dataset_ref.table("posts_questions")

# API request - fetch the table
questions_table = client.get_table(questions_table_ref)

# preview the first five lines of the "posts_questions" table
client.list_rows(questions_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,60472844,How can I generate a list of node features as ...,"<p>For a Graph based machine learning task, I ...",None,0,0,None,2020-03-01 07:17:35.877000+00:00,None,2020-03-01 07:17:35.877000+00:00,None,None,None,None,11623307,None,1,0,arrays|python-3.x|numpy,1
1,60472846,Java script - random word generator,<p>Am curious why is it math.floor returns goo...,None,0,0,None,2020-03-01 07:17:41.333000+00:00,None,2020-03-01 07:17:41.333000+00:00,None,None,None,None,11073690,None,1,0,javascript,1
2,60214527,Monitor Backup age using munin,<p>I have a server with does regular backups e...,None,0,0,None,2020-02-13 18:41:21.397000+00:00,None,2020-02-13 18:41:21.397000+00:00,None,None,None,None,491605,None,1,0,munin,2
3,60223941,Deleting collection item in debug mode,<p>I want to delete items from collection in d...,None,0,0,None,2020-02-14 09:55:48.600000+00:00,None,2020-02-14 09:55:48.600000+00:00,None,None,None,None,10339036,None,1,0,visual-studio|debugging,2
4,60282584,Android Services Scenario questions,<p>I'm new to Android Services and looking out...,None,0,0,None,2020-02-18 13:58:24.037000+00:00,None,2020-02-18 13:58:24.037000+00:00,None,None,None,None,7325003,None,1,0,android,2


Are there any fields that identify what topic or technology each question is about? If so, how could you find the IDs of users who answered questions about a specific topic?

Think about it, and then check the solution by running the code in the next cell.

In [7]:
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
`posts_questions` has a column called `tags` which lists the topics/technologies each question is about.

`posts_answers` has a column called `parent_id` which identifies the ID of the question each answer is responding to.
`posts_answers` also has an `owner_user_id` column which specifies the ID of the user who answered the question.

You can join these two tables to:
- determine the `tags` for each answer, and then
- select the `owner_user_id` of the answers on the desired tag.

This is exactly what you will do over the next few questions.


<hr/>

## 3) Selecting the Right Questions

A lot of this data is text. 

We'll explore one last technique in this course which you can apply to this text.

A **WHERE** clause can limit your results to rows with certain text using the **LIKE** feature. For example, to select just the third row of the `pets` table from the tutorial, we could use the query in the picture below.

![](https://i.imgur.com/RccsXBr.png) 

You can also use `%` as a "wildcard" for any number of characters. So you can also get the third row with:

```
query = """
        SELECT * 
        FROM `bigquery-public-data.pet_records.pets` 
        WHERE Name LIKE '%ipl%'
        """
```

Try this yourself. Write a query that selects the `id`, `title` and `owner_user_id` columns from the `posts_questions` table. 
- Restrict the results to rows that contain the word "bigquery" in the `tags` column. 
- Include rows where there is other text in addition to the word "bigquery" (e.g., if a row has a tag "bigquery-sql", your results should include that too).

In [8]:
# your code here
questions_query = """
                  SELECT id, title, owner_user_id
                  FROM `bigquery-public-data.stackoverflow.posts_questions`
                  WHERE tags LIKE '%bigquery%'
                  """

# set up the query (cancel the query if it would use too much of your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
questions_query_job = client.query(questions_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
questions_results = questions_query_job.to_dataframe()

# preview results
print(questions_results.head())

# check your answer
q_3.check()

         id                                              title  owner_user_id
0  32686348      Does BigQuery support nested CASE statements?      2877278.0
1  32960707      Dataproc + BigQuery examples - any available?      2877278.0
2  32660226              Billing Tier is out of expected range      5221944.0
3  32626848  totalBytesBilled is different from totalBytesP...      2881671.0
4  32619298                      Google bigquery - update sql?      2647042.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [9]:
# for a hint or the solution, uncomment the appropriate line below.
# q_3.hint()
# q_3.solution()

<hr/>

## 4. Your First JOIN

Now that you have a query to select questions on any given topic (in this case, you chose "bigquery"), you can find the answers to those questions with a **JOIN**.  

Write a query that returns the `id`, `body` and `owner_user_id` columns from the `posts_answers` table for answers to "bigquery"-related questions. 
- You should have one row in your results for each answer to a question that has "bigquery" in the tags.  
- Remember you can get the tags for a question from the `tags` column in the `posts_questions` table.

Here's a reminder of what a **JOIN** looked like in the tutorial:
```
query = """
        SELECT p.Name AS Pet_Name, o.Name AS Owner_Name
        FROM `bigquery-public-data.pet_records.pets` as p
        INNER JOIN `bigquery-public-data.pet_records.owners` as o 
            ON p.ID = o.Pet_ID
        """
```

It may be useful to scroll up and review the first several rows of the `posts_answers` and `posts_questions` tables.  

In [10]:
# your code here
answers_query = """
                SELECT a.id, a.body, a.owner_user_id
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS q 
                INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                    ON q.id = a.parent_id
                WHERE q.tags LIKE '%bigquery%'
                """

# set up the query (cancel the query if it would use too much of your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
answers_query_job = client.query(answers_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
answers_results = answers_query_job.to_dataframe()

# preview results
print(answers_results.head())

# check your answer
q_4.check()

         id                                               body  owner_user_id
0  59892927  <p>You can do:</p>\n\n<pre><code>SELECT &lt;FI...     12518487.0
1  59895471  <p>This issue is a little tricky and not so ob...     12518487.0
2  59897438  <p>I fixed this issue. Here's where I went wro...     12620429.0
3  59918014  <p>What I ended up doing was to format the JSO...      7732544.0
4  59924766  <p>Be careful when copying the sample schema t...      7096300.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [11]:
# for a hint or the solution, uncomment the appropriate line below.
# q_4.hint()
# q_4.solution()

<hr/>

## 5. Answer the Question
You have the merge you need. But you want a list of users who have answered many questions... which requires more work beyond your previous result.

Write a new query that has a single row for each user who answered at least one question with a tag that includes the string "bigquery". Your results should have two columns:
- `user_id` - contains the `owner_user_id` column from the `posts_answers` table
- `number_of_answers` - contains the number of answers the user has written to "bigquery"-related questions

In [12]:
# your code here
bigquery_experts_query = """
                         SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
                         FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                         INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                             ON q.id = a.parent_id
                         WHERE q.tags LIKE '%bigquery%'
                         GROUP BY user_id
                         """

# set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
bigquery_experts_query_job = client.query(bigquery_experts_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
bigquery_experts_results = bigquery_experts_query_job.to_dataframe()

# preview results
print(bigquery_experts_results.head())

# Check your answer
q_5.check()

      user_id  number_of_answers
0   5344262.0                  2
1  11869962.0                  1
2   3911997.0                  1
3   7387049.0                  1
4   3226630.0                  7


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [13]:
# for a hint or the solution, uncomment the appropriate line below.
# q_5.hint()
# q_5.solution()

<hr/>

## 6. Building a more Generally Useful Service

How could you convert what you've done to a general function a website could call on the backend to get experts on any topic?  

Think about it and then check the solution below.

In [14]:
q_6.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

def expert_finder(topic, client):
    '''
    Returns a DataFrame with the user IDs who have written Stack Overflow answers on a topic.

    Inputs:
        topic: A string with the topic of interest
        client: A Client object that specifies the connection to the Stack Overflow dataset

    Outputs:
        results: A DataFrame with columns for user_id and number_of_answers. Follows similar logic to bigquery_experts_results shown above.
    '''
    my_query = """
               SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
               FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
               INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                   ON q.id = a.parent_Id
               WHERE q.tags like '%{topic}%'
               GROUP BY a.owner_user_id
               """
               
    # Set up the query (a real service would have good error handling for 
    # queries that scan too much data)
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)      
    my_query_job = client.query(my_query, job_config=safe_config)
    
    # API request - run the query, and return a pandas DataFrame
    results = my_query_job.to_dataframe()

    return results

```

# Congratulations!

You know all the key components to use BigQuery and SQL effectively. Your SQL skills are sufficient to unlock many of the world's largest datasets.

Want to go play with your new powers?  Kaggle has BigQuery datasets available [here](https://www.kaggle.com/datasets?sortBy=hottest&group=public&page=1&pageSize=20&size=sizeAll&filetype=fileTypeBigQuery).

# Feedback

Bring any questions or feedback to the [Learn Discussion Forum](https://www.kaggle.com/learn-forum).

---
**[SQL Home Page](https://www.kaggle.com/learn/intro-to-sql)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum) to chat with other Learners.*